In [1]:
import sys
import os
import json

import pandas as pd
import numpy as np

In [2]:
root_dir = 'C:/Users/delst/OneDrive/Desktop/Code/Workspace/JSON_to_PGSQL'
sys.path.append(root_dir)

from A_Main.Configurations.setup_env import setup_environment
config = setup_environment(root_dir)

In [3]:
with open(config.raw_data, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

In [4]:
sdo_parq = config.sdo_parq
sdo_feather = config.sdo_feather

---

# **Select Properties**

In [6]:
data = raw_data.get('data', {})
posts = data.get('children', [])
post_data = posts[2].get('data', {})
media_data = post_data.get('media', {})
reddit_video_data = media_data.get('reddit_video', {})
reddit_video_data

{'bitrate_kbps': 1200,
 'fallback_url': 'https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?source=fallback',
 'has_audio': True,
 'height': 480,
 'width': 854,
 'scrubber_media_url': 'https://v.redd.it/evmp79fbr56b1/DASH_96.mp4',
 'dash_url': 'https://v.redd.it/evmp79fbr56b1/DASHPlaylist.mpd?a=1689435086%2CZGI5YzZlMTBkMWViZmZjM2E4NjZhODJlMTM3ZjVjMWNhYWI3NjM5YTcyZmE5ZjA2Mjk5OGUyZjQ5OWY3MDdiMQ%3D%3D&amp;v=1&amp;f=sd',
 'duration': 68,
 'hls_url': 'https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3u8?a=1689435086%2CMzA1YjFkMDUwYTIwZDkzZTZjMWVjMGZhYTJmNmI4N2Y0ZWJkYjQ1MjU2YzgwZDJiNmI4MmU0OWRhYTU4MGIyMQ%3D%3D&amp;v=1&amp;f=sd',
 'is_gif': False,
 'transcoding_status': 'completed'}

---

# **Post Data**

In [7]:
post_data_keys = set()

for post in posts:
    post_data = post.get('data', {})
    
    for key in post_data.keys():
        post_data_keys.add(key)
post_data_keys

{'all_awardings',
 'allow_live_comments',
 'approved_at_utc',
 'approved_by',
 'archived',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_is_blocked',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'banned_at_utc',
 'banned_by',
 'can_gild',
 'can_mod_post',
 'category',
 'clicked',
 'content_categories',
 'contest_mode',
 'created',
 'created_utc',
 'discussion_type',
 'distinguished',
 'domain',
 'downs',
 'edited',
 'gallery_data',
 'gilded',
 'gildings',
 'hidden',
 'hide_score',
 'id',
 'is_created_from_ads_ui',
 'is_crosspostable',
 'is_gallery',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'likes',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',


In [8]:
post_data_store = []

for post in posts:
    post_data = post.get('data', {})
    data_list = []
    
    for key in post_data_keys:
        data_list.append(post_data.get(key, np.nan))
    
    post_data_store.append(data_list)

In [9]:
df_post_data = pd.DataFrame(post_data_store, columns=list(post_data_keys))
df_post_data.head()

,suggested_sort,media,link_flair_richtext,archived,thumbnail,contest_mode,author_flair_richtext,url,author_flair_text,domain,...,name,content_categories,mod_reason_by,thumbnail_width,url_overridden_by_dest,whitelist_status,link_flair_text_color,parent_whitelist_status,stickied,upvote_ratio
0,None,None,"[{'e': 'text', 't': 'Clubhouse'}]",False,https://b.thumbs.redditmedia.com/hv0ShtFGoPsuv...,False,[],https://i.redd.it/urxp0hrc666b1.png,None,i.redd.it,...,t3_14a0ayt,None,None,140.0,https://i.redd.it/urxp0hrc666b1.png,None,dark,None,False,0.96
1,confidence,None,[],False,https://b.thumbs.redditmedia.com/E5JI3dtZeIHgA...,False,[],https://i.redd.it/u9e31qnjg66b1.jpg,None,i.redd.it,...,t3_14a1hq5,None,None,140.0,https://i.redd.it/u9e31qnjg66b1.jpg,some_ads,light,some_ads,False,0.96
2,None,"{'reddit_video': {'bitrate_kbps': 1200, 'fallb...","[{'e': 'text', 't': 'Video/Gif'}]",False,https://external-preview.redd.it/NndmejhuYmJyN...,False,[],https://v.redd.it/evmp79fbr56b1,None,v.redd.it,...,t3_149yrlt,None,None,140.0,https://v.redd.it/evmp79fbr56b1,some_ads,dark,some_ads,False,0.95
3,None,None,[],False,https://b.thumbs.redditmedia.com/UxhMsrSge7gz2...,False,[],https://www.independent.co.uk/news/world/ameri...,None,independent.co.uk,...,t3_14a064k,None,None,140.0,https://www.independent.co.uk/news/world/ameri...,all_ads,dark,all_ads,False,0.96
4,confidence,"{'reddit_video': {'bitrate_kbps': 2400, 'fallb...",[],False,https://b.thumbs.redditmedia.com/fA6DRdiu3L0uh...,False,[],https://v.redd.it/9az2zw4ou56b1,None,v.redd.it,...,t3_149z3aw,None,None,140.0,https://v.redd.it/9az2zw4ou56b1,all_ads,light,all_ads,False,0.91


---

# **Video Data**

In [10]:
reddit_video_keys = set()
for post in posts:
    post_data = post.get('data', {})
    media_data = post_data.get('media', {})
    
    if media_data != None:
        reddit_video = media_data.get('reddit_video', {})
        reddit_video_keys.update(reddit_video.keys())
reddit_video_keys = sorted(list(reddit_video_keys))

In [11]:
video_data_store = []

for post in posts:
    post_data = post.get('data', {})
    post_id = post_data.get('id', None)
    
    media_data = post_data.get('media', {})
    
    if media_data != None:
        reddit_video = media_data.get('reddit_video', {})
        
        # Executes if dict is not empty or is not false
        if reddit_video:
            video_data = [post_id] + [reddit_video.get(key) for key in reddit_video_keys]
            video_data_store.append(video_data)
        
        # Execute if a dict {}
        # if isinstance(reddit_video, dict):
        #     single_data = [post_id] + [reddit_video.get(key) for key in reddit_video_keys]
        #     video_data_store.append(single_data)

In [12]:
df_video_data = pd.DataFrame(video_data_store, columns=['post_id'] + reddit_video_keys)
df_video_data.head()

,post_id,bitrate_kbps,dash_url,duration,fallback_url,has_audio,height,hls_url,is_gif,scrubber_media_url,transcoding_status,width
0,149yrlt,1200,https://v.redd.it/evmp79fbr56b1/DASHPlaylist.m...,68,https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?s...,True,480,https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3...,False,https://v.redd.it/evmp79fbr56b1/DASH_96.mp4,completed,854
1,149z3aw,2400,https://v.redd.it/9az2zw4ou56b1/DASHPlaylist.m...,58,https://v.redd.it/9az2zw4ou56b1/DASH_720.mp4?s...,None,720,https://v.redd.it/9az2zw4ou56b1/HLSPlaylist.m3...,False,https://v.redd.it/9az2zw4ou56b1/DASH_96.mp4,completed,406
2,149wwzb,2400,https://v.redd.it/vstcrxxm756b1/DASHPlaylist.m...,178,https://v.redd.it/vstcrxxm756b1/DASH_720.mp4?s...,None,720,https://v.redd.it/vstcrxxm756b1/HLSPlaylist.m3...,False,https://v.redd.it/vstcrxxm756b1/DASH_96.mp4,completed,406
3,14a36xk,2400,https://v.redd.it/regbdl8au66b1/DASHPlaylist.m...,43,https://v.redd.it/regbdl8au66b1/DASH_720.mp4?s...,False,720,https://v.redd.it/regbdl8au66b1/HLSPlaylist.m3...,True,https://v.redd.it/regbdl8au66b1/DASH_96.mp4,completed,1280
4,149zpnd,1200,https://v.redd.it/0zpvfvyk066b1/DASHPlaylist.m...,354,https://v.redd.it/0zpvfvyk066b1/DASH_480.mp4?s...,True,394,https://v.redd.it/0zpvfvyk066b1/HLSPlaylist.m3...,False,https://v.redd.it/0zpvfvyk066b1/DASH_96.mp4,completed,854


---

# **Image Data**

In [14]:
image_data_keys = set()
image_source_keys = set()
image_resolutions_keys = set()

for post in posts:
    post_data = post.get('data', {})
    preview_data = post_data.get('preview', {})
    
    if preview_data != None:
        image_data = preview_data.get('images', [])

        for image in image_data:
            
            data_keys = image.keys()
            source_keys = image.get('source', {}).keys()
            
            image_data_keys.update(data_keys)
            image_source_keys.update(source_keys)
            
            for i in image.get('resolutions', []):
                res_keys = i.keys()
                image_resolutions_keys.update(res_keys)
    
image_data_keys = image_data_keys - {'resolutions', 'source'}

image_data_keys = sorted(list(image_data_keys))
image_source_keys = sorted(list(image_source_keys))
image_resolutions_keys = sorted(list(image_resolutions_keys))

In [15]:
image_data_store = sorted([])
image_source_store = sorted([])
image_resolution_store = sorted([])

for post in posts:
    post_data = post.get('data', {})
    post_id = post_data.get('id', None)
    
    preview_data = post_data.get('preview', {})
    
    if preview_data is not None:
        images = preview_data.get('images', [])
        
        for image in images:
            image_id = image.get('id', None)
            
            # Collect image data
            image_data = [post_id] + [image.get(key) for key in image_data_keys]
            image_data_store.append(image_data)
            
            # Collect source data
            source = image.get('source', {})
            if source:
                source_data = [post_id, image_id] + [source.get(key) for key in image_source_keys]
                image_source_store.append(source_data)
            
            # Collect resolution data
            # resolution = image.get('resolutions', [])
            # if resolution:
            #     resolution_data = [post_id, image_id] + [i.get(key) for key in image_resolutions_keys for i in resolution]
            #     image_resolution_store.append(resolution_data)
                
            # Collect resolution data for each image
            for item in image.get('resolutions', []):
                res_data = [
                    post_id,
                    image_id,
                    item.get('height'),
                    item.get('url'),
                    item.get('width'),
                ]
                image_resolution_store.append(res_data)

In [16]:
df_image_data = pd.DataFrame(image_data_store, columns=['post_id'] + image_data_keys)
df_image_source = pd.DataFrame(image_source_store, columns=['post_id','id'] + image_source_keys)
df_image_resolution = pd.DataFrame(image_resolution_store, columns=['post_id','id'] + image_resolutions_keys)

In [17]:
display(df_image_data.head())
display(df_image_source.head())
display(df_image_resolution)

,post_id,id,variants
0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,{}
1,14a1hq5,gAk-dl1pjVUL88HkOOUMiT7PqP6wZncsweXFa-i6fUI,{}
2,149yrlt,NndmejhuYmJyNTZiMb0UmIeULhm5TqEhpHt9vw3gmBGF49...,{}
3,14a064k,YQCgKRT3PwoUJd7UixJDP4_jcfVx1QEnV0W2TQH8uPo,{}
4,149z3aw,_UjbEUYm4O61zhRgxj2pPo4GfL-JF2P5N7mdZq9pNTE,{}


,post_id,id,height,url,width
0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1628,https://preview.redd.it/urxp0hrc666b1.png?auto...,1025
1,14a1hq5,gAk-dl1pjVUL88HkOOUMiT7PqP6wZncsweXFa-i6fUI,1306,https://preview.redd.it/u9e31qnjg66b1.jpg?auto...,1046
2,149yrlt,NndmejhuYmJyNTZiMb0UmIeULhm5TqEhpHt9vw3gmBGF49...,540,https://external-preview.redd.it/NndmejhuYmJyN...,960
3,14a064k,YQCgKRT3PwoUJd7UixJDP4_jcfVx1QEnV0W2TQH8uPo,800,https://external-preview.redd.it/46hLWMpB_3Nh4...,1200
4,149z3aw,_UjbEUYm4O61zhRgxj2pPo4GfL-JF2P5N7mdZq9pNTE,1024,https://external-preview.redd.it/WcRhmEDaOB-IJ...,576


,post_id,id,height,url,width
0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,171,https://preview.redd.it/urxp0hrc666b1.png?widt...,108
1,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,343,https://preview.redd.it/urxp0hrc666b1.png?widt...,216
2,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,508,https://preview.redd.it/urxp0hrc666b1.png?widt...,320
3,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1016,https://preview.redd.it/urxp0hrc666b1.png?widt...,640
4,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCayg,1524,https://preview.redd.it/urxp0hrc666b1.png?widt...,960
...,...,...,...,...,...
102,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,193,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,216
103,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,287,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,320
104,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,574,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,640
105,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_Zo,862,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,960


---

In [18]:
df_dict = {
    'post_data': df_post_data,
    'video_data': df_video_data,
    'image_data': df_image_data,
    'image_source': df_image_source,
    'image_resolution': df_image_resolution
}

# save to .feather
for filename, df in df_dict.items():
    df.to_feather(os.path.join(sdo_feather, filename + '.feather'))

In [19]:
df_names = ['post_data', 'video_data', 'image_data', 'image_source', 'image_resolution']
df_dict = {}

for df_name in df_names:
    df_dict[df_name] = pd.read_feather(os.path.join(sdo_feather, df_name + '.feather'))

df_post_data = df_dict['post_data']
df_video_data = df_dict['video_data']
df_image_data = df_dict['image_data']
df_image_source = df_dict['image_source']
df_image_resolution = df_dict['image_resolution']

In [20]:
df_post_data

,suggested_sort,media,link_flair_richtext,archived,thumbnail,contest_mode,author_flair_richtext,url,author_flair_text,domain,...,name,content_categories,mod_reason_by,thumbnail_width,url_overridden_by_dest,whitelist_status,link_flair_text_color,parent_whitelist_status,stickied,upvote_ratio
0,None,None,"[{'a': None, 'e': 'text', 't': 'Clubhouse', 'u...",False,https://b.thumbs.redditmedia.com/hv0ShtFGoPsuv...,False,[],https://i.redd.it/urxp0hrc666b1.png,None,i.redd.it,...,t3_14a0ayt,None,None,140.0,https://i.redd.it/urxp0hrc666b1.png,None,dark,None,False,0.96
1,confidence,None,[],False,https://b.thumbs.redditmedia.com/E5JI3dtZeIHgA...,False,[],https://i.redd.it/u9e31qnjg66b1.jpg,None,i.redd.it,...,t3_14a1hq5,None,None,140.0,https://i.redd.it/u9e31qnjg66b1.jpg,some_ads,light,some_ads,False,0.96
2,None,"{'reddit_video': {'bitrate_kbps': 1200, 'dash_...","[{'a': None, 'e': 'text', 't': 'Video/Gif', 'u...",False,https://external-preview.redd.it/NndmejhuYmJyN...,False,[],https://v.redd.it/evmp79fbr56b1,None,v.redd.it,...,t3_149yrlt,None,None,140.0,https://v.redd.it/evmp79fbr56b1,some_ads,dark,some_ads,False,0.95
3,None,None,[],False,https://b.thumbs.redditmedia.com/UxhMsrSge7gz2...,False,[],https://www.independent.co.uk/news/world/ameri...,None,independent.co.uk,...,t3_14a064k,None,None,140.0,https://www.independent.co.uk/news/world/ameri...,all_ads,dark,all_ads,False,0.96
4,confidence,"{'reddit_video': {'bitrate_kbps': 2400, 'dash_...",[],False,https://b.thumbs.redditmedia.com/fA6DRdiu3L0uh...,False,[],https://v.redd.it/9az2zw4ou56b1,None,v.redd.it,...,t3_149z3aw,None,None,140.0,https://v.redd.it/9az2zw4ou56b1,all_ads,light,all_ads,False,0.91
5,confidence,None,[],False,https://b.thumbs.redditmedia.com/dVTEUorEjpD-f...,False,[],https://i.redd.it/hpo5fd3cv56b1.jpg,None,i.redd.it,...,t3_149z5hi,None,None,140.0,https://i.redd.it/hpo5fd3cv56b1.jpg,all_ads,dark,all_ads,False,0.95
6,None,None,[],False,https://b.thumbs.redditmedia.com/Ix2e1mrlNT1Q9...,False,[],https://www.independent.co.uk/tech/reddit-blac...,None,independent.co.uk,...,t3_149zmar,None,None,140.0,https://www.independent.co.uk/tech/reddit-blac...,all_ads,dark,all_ads,False,0.92
7,None,"{'reddit_video': {'bitrate_kbps': 2400, 'dash_...",[],False,https://external-preview.redd.it/OWw3ZGthdW03N...,False,[],https://v.redd.it/vstcrxxm756b1,None,v.redd.it,...,t3_149wwzb,None,None,140.0,https://v.redd.it/vstcrxxm756b1,some_ads,light,some_ads,False,0.94
8,None,None,[],False,https://b.thumbs.redditmedia.com/d2i8OGKnZPQZ-...,False,[],https://i.redd.it/hb4h956dd56b1.png,None,i.redd.it,...,t3_149xfsx,[comics],None,140.0,https://i.redd.it/hb4h956dd56b1.png,all_ads,dark,all_ads,False,0.88
9,None,"{'reddit_video': {'bitrate_kbps': 2400, 'dash_...",[],False,https://b.thumbs.redditmedia.com/QeyeHriekTJ49...,False,[],https://v.redd.it/regbdl8au66b1,None,v.redd.it,...,t3_14a36xk,None,None,140.0,https://v.redd.it/regbdl8au66b1,all_ads,dark,all_ads,False,0.96


----